# 03. Data Update & Quality Assurance (v2)
## 목표
1. 전체 기간 데이터를 수정주가(Adjusted Close) 기준으로 재수집
2. 티커 메타데이터(섹터/산업) 최신화
3. 품질 검증 및 데이터 정제
4. 파생변수 계산 및 저장

---

## Section 1. 환경 설정 & 티커 목록 로드

In [38]:
# Library Imports
import pandas as pd
import numpy as np
import yfinance as yf
import os
import json
import time
import logging
import shutil
from pathlib import Path
from datetime import datetime, timedelta
from tqdm import tqdm
import warnings

warnings.filterwarnings('ignore')

# 시각화 설정
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

In [39]:
# 경로 설정
PROJECT_ROOT = Path('.').resolve()
DATA_DIR = PROJECT_ROOT / 'Data_set'
QA_DIR = DATA_DIR / 'QA'
LOG_DIR = DATA_DIR / 'Logs'
TEMP_DIR = DATA_DIR / 'temp_batches'

# 디렉토리 생성
for directory in [DATA_DIR, QA_DIR, LOG_DIR, TEMP_DIR]:
    directory.mkdir(exist_ok=True)

print(f"  작업 디렉토리: {PROJECT_ROOT}")
print(f"  데이터 디렉토리: {DATA_DIR}")

  작업 디렉토리: /Users/yu_seok/Documents/workspace/nbCamp/Project/Yahoo Finance
  데이터 디렉토리: /Users/yu_seok/Documents/workspace/nbCamp/Project/Yahoo Finance/Data_set


In [40]:
# 로깅 설정
log_file = LOG_DIR / f'data_update_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file, encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

logger.info("=" * 60)
logger.info("데이터 업데이트 프로세스 시작")
logger.info("=" * 60)
print(f"  로그 파일: {log_file}")

2026-01-13 13:09:39,482 - INFO - ============================================================
2026-01-13 13:09:39,482 - INFO - 데이터 업데이트 프로세스 시작
2026-01-13 13:09:39,483 - INFO - ============================================================


  로그 파일: /Users/yu_seok/Documents/workspace/nbCamp/Project/Yahoo Finance/Data_set/Logs/data_update_20260113_130939.log


In [41]:
# 이전 작업 재개 기능
def check_previous_work():
    """이전 작업 상태 확인 및 재개 옵션 제공"""
    batch_files = list(TEMP_DIR.glob('batch_*.csv'))
    
    if batch_files:
        print("\n" + "=" * 60)
        print("  이전 작업 발견")
        print("=" * 60)
        print(f"발견된 배치 파일: {len(batch_files)}개")
        print(f"위치: {TEMP_DIR}")
        
        # 자동으로 이어서 진행
        response = input("\n이전 작업을 이어서 진행하시겠습니까? (y/n) [y]: ").strip().lower()
        
        if response in ['', 'y', 'yes']:
            logger.info(f"이전 작업 재개 - {len(batch_files)}개 배치 발견")
            print(" 이전 작업을 이어서 진행합니다.")
            return True
        else:
            print("\n기존 임시 파일을 삭제하고 새로 시작합니다")
            shutil.rmtree(TEMP_DIR)
            TEMP_DIR.mkdir()
            logger.info("임시 파일 삭제 후 새로 시작")
            print(" 새로운 작업을 시작합니다.")
            return False
    else:
        logger.info("새로운 작업 시작")
        print(" 새로운 작업을 시작합니다.")
        return False

RESUME_MODE = check_previous_work()

2026-01-13 13:09:39,490 - INFO - 새로운 작업 시작


 새로운 작업을 시작합니다.


In [42]:
# 티커 목록 로드
print("\n" + "=" * 60)
print("티커 목록 로드")
print("=" * 60)

master_file = DATA_DIR / 'stock_daily_master.csv'

if not master_file.exists():
    logger.error(f"파일을 찾을 수 없습니다: {master_file}")
    raise FileNotFoundError(f"파일이 존재하지 않습니다: {master_file}")

try:
    df_old = pd.read_csv(master_file)
    logger.info(f"기존 파일 로드 완료: {master_file}")
    
    tickers = (
        df_old['Company']
        .dropna()
        .astype(str)
        .str.strip()
        .loc[lambda s: s.ne('') & s.ne('nan')]
        .unique()
        .tolist()
    )
    tickers = sorted(tickers)
    
    print(f"  총 티커 수: {len(tickers)}")
    print(f"  샘플 (처음 20개): {tickers[:20]}")
    logger.info(f"티커 목록 로드 완료: {len(tickers)}개")
    
    # 메모리 해제
    del df_old
    
except Exception as e:
    logger.error(f"티커 목록 로드 실패: {str(e)}")
    raise


티커 목록 로드


2026-01-13 13:09:41,700 - INFO - 기존 파일 로드 완료: /Users/yu_seok/Documents/workspace/nbCamp/Project/Yahoo Finance/Data_set/stock_daily_master.csv
2026-01-13 13:09:41,831 - INFO - 티커 목록 로드 완료: 481개


  총 티커 수: 481
  샘플 (처음 20개): ['A', 'AAPL', 'ABBV', 'ABEV', 'ABNB', 'ABT', 'ACGL', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEM', 'AEP', 'AFL', 'AIG', 'AJG', 'ALC']


## Section 2. 전체 기간 데이터 재수집 (수정주가 반영)

In [ ]:
def fetch_adjusted_price_data(ticker_list, start_date, end_date, batch_size=10, max_retries=3):
    """
    yfinance 데이터 수집 함수
    
    Args:
        ticker_list: 티커 목록
        start_date: 시작일 (YYYY-MM-DD)
        end_date: 종료일 (YYYY-MM-DD)
        batch_size: 배치 크기 (기본값: 10)
        max_retries: 최대 재시도 횟수 (기본값: 3)
    
    Returns:
        DataFrame: 수정주가 반영된 OHLCV 데이터
        list: 실패한 티커 목록
    """
    all_data = []
    failed_tickers = []
    
    # 배치 분할
    chunks = [ticker_list[i:i + batch_size] for i in range(0, len(ticker_list), batch_size)]
    logger.info(f"총 {len(chunks)}개 배치로 분할 (배치 크기: {batch_size})")
    
    for chunk_idx, chunk in enumerate(tqdm(chunks, desc=" 데이터 수집 중")):
        # 이미 저장된 배치가 있으면 건너뛰기
        temp_file = TEMP_DIR / f'batch_{chunk_idx:04d}.csv'
        if temp_file.exists():
            try:
                df_batch = pd.read_csv(temp_file, parse_dates=['Date'])
                all_data.append(df_batch)
                logger.info(f"배치 {chunk_idx}/{len(chunks)} - 캐시에서 로드됨 ({len(df_batch)} 행)")
                print(f"    배치 {chunk_idx} 캐시에서 로드")
                continue
            except Exception as e:
                logger.warning(f"배치 {chunk_idx} 캐시 로드 실패: {str(e)}, 재수집 시도")
        
        tickers_str = ' '.join(chunk)
        batch_success = False
        
        # Exponential backoff 재시도
        for attempt in range(max_retries):
            try:
                # 배치 다운로드
                data = yf.download(
                    tickers_str,
                    start=start_date,
                    end=end_date,
                    group_by='ticker',
                    auto_adjust=True,
                    progress=False,
                    threads=False  # Rate limit 방지를 위해 단일 스레드 사용
                )
                
                # 데이터가 비어있는지 확인
                if data.empty:
                    logger.warning(f"배치 {chunk_idx} - 데이터 없음")
                    failed_tickers.extend(chunk)
                    break
                
                # 데이터 처리
                batch_data = []
                
                if len(chunk) == 1:
                    # 단일 티커 처리
                    symbol = chunk[0]
                    if not data.empty:
                        df_ticker = data.copy().reset_index()
                        
                        # 컬럼명 정규화
                        if 'index' in df_ticker.columns and 'Date' not in df_ticker.columns:
                            df_ticker = df_ticker.rename(columns={'index': 'Date'})
                        
                        df_ticker['Company'] = symbol
                        
                        # Close 값이 있는 행만 유지
                        if 'Close' in df_ticker.columns:
                            df_ticker = df_ticker.dropna(subset=['Close'])
                            if not df_ticker.empty:
                                batch_data.append(df_ticker)
                            else:
                                failed_tickers.append(symbol)
                        else:
                            failed_tickers.append(symbol)
                    else:
                        failed_tickers.append(symbol)
                else:
                    # 멀티 티커 처리
                    for symbol in chunk:
                        try:
                            # 티커 데이터 추출
                            if hasattr(data.columns, 'levels'):
                                # MultiIndex인 경우
                                if symbol in data.columns.get_level_values(0):
                                    df_ticker = data[symbol].copy()
                                else:
                                    failed_tickers.append(symbol)
                                    continue
                            else:
                                # 단순 Index인 경우
                                df_ticker = data.copy()
                            
                            # Date를 컬럼으로 변환
                            df_ticker = df_ticker.reset_index()
                            
                            # 컬럼명 정규화
                            if 'index' in df_ticker.columns and 'Date' not in df_ticker.columns:
                                df_ticker = df_ticker.rename(columns={'index': 'Date'})
                            
                            df_ticker['Company'] = symbol
                            
                            # Close 값이 있는 행만 유지
                            if 'Close' in df_ticker.columns:
                                df_ticker = df_ticker.dropna(subset=['Close'])
                                if not df_ticker.empty:
                                    batch_data.append(df_ticker)
                                else:
                                    failed_tickers.append(symbol)
                            else:
                                failed_tickers.append(symbol)
                        
                        except Exception as e:
                            logger.error(f"티커 {symbol} 처리 실패: {str(e)}")
                            failed_tickers.append(symbol)
                
                # 배치 데이터 저장
                if batch_data:
                    df_batch = pd.concat(batch_data, ignore_index=True)
                    
                    # Date 컬럼 처리
                    if 'Date' in df_batch.columns:
                        df_batch['Date'] = pd.to_datetime(df_batch['Date'])
                        if df_batch['Date'].dt.tz is not None:
                            df_batch['Date'] = df_batch['Date'].dt.tz_localize(None)
                    
                    # 임시 파일로 저장
                    df_batch.to_csv(temp_file, index=False)
                    all_data.append(df_batch)
                    
                    logger.info(f"배치 {chunk_idx}/{len(chunks)} 성공 - {len(df_batch)} 행 저장")
                    print(f"     배치 {chunk_idx} 완료 ({len(df_batch)} 행)")
                    batch_success = True
                else:
                    logger.warning(f"배치 {chunk_idx} - 처리된 데이터 없음")
                
                break  # 성공 시 재시도 루프 종료
            
            except Exception as e:
                error_str = str(e)
                
                # Rate Limit 에러 감지
                if 'Rate' in error_str or '429' in error_str or 'Too Many Requests' in error_str:
                    wait_time = (2 ** attempt) * 2  # 2초, 4초, 8초
                    logger.warning(f"배치 {chunk_idx} - {wait_time}초 대기 (시도 {attempt+1}/{max_retries})")
                    print(f"   Rate Limit 감지, {wait_time}초 대기 중")
                    time.sleep(wait_time)
                else:
                    logger.error(f"배치 {chunk_idx} 에러: {error_str}")
                    if attempt == max_retries - 1:
                        failed_tickers.extend(chunk)
                    break
        
        if not batch_success and attempt == max_retries - 1:
            logger.error(f"배치 {chunk_idx} 최종 실패")
        
        # 배치 간 대기 (Rate limit 방지)
        time.sleep(2)
    
    # 데이터 병합
    if all_data:
        logger.info(f"총 {len(all_data)}개 배치 데이터 병합 중")
        df_combined = pd.concat(all_data, ignore_index=True)
        
        # Date 컬럼 최종 처리
        if 'Date' in df_combined.columns:
            df_combined['Date'] = pd.to_datetime(df_combined['Date'])
            if df_combined['Date'].dt.tz is not None:
                df_combined['Date'] = df_combined['Date'].dt.tz_localize(None)
        
        logger.info(f"최종 데이터: {df_combined.shape}")
        logger.info(f"컬럼: {df_combined.columns.tolist()}")
        
        return df_combined, failed_tickers
    else:
        logger.error("수집된 데이터가 없습니다")
        return pd.DataFrame(), failed_tickers

In [44]:
# 전체 기간 재수집 실행
print("\n" + "=" * 60)
print("전체 기간 데이터 재수집 (수정주가 반영)")
print("=" * 60)

# 수집 기간 설정
start_date = '2018-11-29'
end_date = datetime.now().strftime('%Y-%m-%d')

print(f"기간: {start_date} ~ {end_date}")
print(f"티커 수: {len(tickers)}")
print(f"예상 소요 시간: ~{len(tickers) * 0.5 / 60:.1f}분\n")

logger.info(f"데이터 수집 시작 - 기간: {start_date} ~ {end_date}, 티커 수: {len(tickers)}")

# 재수집 실행
df_raw, failed = fetch_adjusted_price_data(
    tickers, 
    start_date, 
    end_date, 
    batch_size=10,
    max_retries=3
)

print("\n" + "=" * 60)
print("수집 결과")
print("=" * 60)
print(f"  성공: {len(tickers) - len(failed)}개")
print(f" 실패: {len(failed)}개")

if failed:
    print(f"\n실패 티커 (처음 20개): {failed[:20]}")
    logger.warning(f"실패 티커: {len(failed)}개 - {failed[:50]}")

# 결과 확인
if not df_raw.empty:
    print(f"\n  데이터 수집 성공")
    print(f"  Shape: {df_raw.shape}")
    print(f"  컬럼: {df_raw.columns.tolist()}")
    print(f"  기간: {df_raw['Date'].min()} ~ {df_raw['Date'].max()}")
    print(f"  기업 수: {df_raw['Company'].nunique()}")
    
    # 필수 컬럼 확인
    required_cols = ['Date', 'Company', 'Close']
    for col in required_cols:
        print(f"  - {col}: {' ' if col in df_raw.columns else '   '}")
    
    logger.info(f"수집 성공 - Shape: {df_raw.shape}, 기업 수: {df_raw['Company'].nunique()}")
else:
    print("\n    데이터가 비어있습니다.")
    logger.error("수집된 데이터가 없습니다")

2026-01-13 13:09:41,869 - INFO - 데이터 수집 시작 - 기간: 2018-11-29 ~ 2026-01-13, 티커 수: 481
2026-01-13 13:09:41,870 - INFO - 총 49개 배치로 분할 (배치 크기: 10)



전체 기간 데이터 재수집 (수정주가 반영)
기간: 2018-11-29 ~ 2026-01-13
티커 수: 481
예상 소요 시간: ~4.0분



 데이터 수집 중:   0%|          | 0/49 [00:00<?, ?it/s]2026-01-13 13:09:44,936 - INFO - 배치 0/49 성공 - 17369 행 저장


     배치 0 완료 (17369 행)


 데이터 수집 중:   2%|▏         | 1/49 [00:05<04:02,  5.06s/it]2026-01-13 13:09:49,917 - INFO - 배치 1/49 성공 - 17792 행 저장


...

     배치 46 완료 (17880 행)


 데이터 수집 중:  96%|█████████▌| 47/49 [03:42<00:09,  4.65s/it]2026-01-13 13:13:27,539 - INFO - 배치 47/49 성공 - 17785 행 저장


     배치 47 완료 (17785 행)


 데이터 수집 중: 100%|██████████| 49/49 [03:50<00:00,  4.69s/it]
2026-01-13 13:13:31,896 - INFO - 총 48개 배치 데이터 병합 중...
2026-01-13 13:13:31,926 - INFO - 최종 데이터: (843513, 7)
2026-01-13 13:13:31,927 - INFO - 컬럼: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Company']
2026-01-13 13:13:31,949 - WARNING - 실패 티커: 1개 - ['ZS']
2026-01-13 13:13:31,983 - INFO - 수집 성공 - Shape: (843513, 7), 기업 수: 480



수집 결과
  성공: 480개
 실패: 1개

실패 티커 (처음 20개): ['ZS']

  데이터 수집 성공
  Shape: (843513, 7)
  컬럼: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Company']
  기간: 2018-11-29 00:00:00 ~ 2026-01-12 00:00:00
  기업 수: 480
  - Date:  
  - Company:  
  - Close:  


In [45]:
# 실패한 티커 재시도
if failed and len(failed) > 0:
    print("\n" + "=" * 60)
    print("실패한 티커 재수집 시도")
    print("=" * 60)
    print(f"재시도할 티커 수: {len(failed)}")
    print(f"더 작은 배치 크기(5)로 재시도 중\n")
    
    logger.info(f"실패 티커 재시도 시작: {len(failed)}개")
    
    # 5초 대기 후 재시도
    time.sleep(5)
    
    df_retry, failed_final = fetch_adjusted_price_data(
        failed, 
        start_date, 
        end_date, 
        batch_size=5,   # 더 작은 배치
        max_retries=5   # 더 많은 재시도
    )
    
    # --- 재시도 병합 가드 (캐시 중복 방지) ---
    def _normalize_tickers(tickers):
        """티커 목록 정규화 (대문자, 공백 제거)"""
        if tickers is None:
            return []
        return [str(t).strip().upper() for t in tickers if str(t).strip()]
    
    if df_retry is None or df_retry.empty:
        logger.info("재수집 결과가 비어 있음: df_retry empty -> 병합 생략")
        print(f"\n    재수집 실패")
    else:
        # 티커 목록 정규화
        failed_norm = _normalize_tickers(failed)
        failed_final_norm = _normalize_tickers(failed_final)
        
        retry_requested = set(failed_norm)
        retry_failed = set(failed_final_norm)
        retry_succeeded = retry_requested - retry_failed
        
        logger.info(
            f"재수집 요약 - 요청:{len(retry_requested)} 성공:{len(retry_succeeded)} 실패:{len(retry_failed)}"
        )
        
        if not retry_succeeded:
            logger.warning(
                f"재수집 성공 티커가 없음 -> df_retry 병합 생략 (요청={sorted(retry_requested)}, 실패={sorted(retry_failed)})"
            )
            print(f"\n    재수집 실패 (모든 티커 실패)")
        else:
            # 티커 컬럼 식별
            candidate_cols = ["Company", "Ticker", "Symbol"]
            ticker_col = next((c for c in candidate_cols if c in df_retry.columns), None)
            
            if ticker_col is None:
                raise KeyError(
                    f"df_retry에 티커 컬럼이 없음. 기대 컬럼 후보={candidate_cols}, 실제 컬럼={list(df_retry.columns)}"
                )
            
            # 티커 컬럼 정규화 (복사본에서 작업)
            df_retry_work = df_retry.copy()
            df_retry_work[ticker_col] = df_retry_work[ticker_col].astype(str).str.strip().str.upper()
            
            # 예상치 못한 티커 경고 (캐시 오염 감지)
            retry_returned_tickers = set(df_retry_work[ticker_col].dropna().unique())
            unexpected = retry_returned_tickers - retry_succeeded
            if unexpected:
                logger.warning(
                    f"재수집 df에 예상치 못한 티커가 포함됨 (캐시 오염/배치 재사용 가능). "
                    f"예상={len(retry_succeeded)}개, 실제={len(retry_returned_tickers)}개, "
                    f"unexpected(sample)={sorted(list(unexpected))[:20]}"
                )
            
            # 실제 성공한 티커만 필터링
            df_retry_filtered = df_retry_work[df_retry_work[ticker_col].isin(retry_succeeded)]
            
            if df_retry_filtered.empty:
                logger.warning(
                    f"재수집 성공 티커({sorted(list(retry_succeeded))})에 해당하는 행이 df_retry에 없음 -> 병합 생략"
                )
                print(f"\n    재수집 데이터 없음 (필터링 후 빈 결과)")
            else:
                pre_rows = len(df_raw)
                add_rows = len(df_retry_filtered)
                
                # 병합 실행
                df_raw = pd.concat([df_raw, df_retry_filtered], ignore_index=True)
                
                logger.info(
                    f"재수집 데이터 병합 완료: +{add_rows} 행 (이전={pre_rows}, 이후={len(df_raw)})"
                )
                print(f"\n  재수집 성공: {len(retry_succeeded)}개 티커, +{add_rows:,} 행")
                
                # 중복 제거 가드 (안전장치)
                dedupe_keys = []
                if ticker_col in df_raw.columns:
                    dedupe_keys.append(ticker_col)
                if "Date" in df_raw.columns:
                    dedupe_keys.append("Date")
                
                if len(dedupe_keys) >= 2:
                    before = len(df_raw)
                    df_raw = df_raw.drop_duplicates(subset=dedupe_keys, keep="last").reset_index(drop=True)
                    removed = before - len(df_raw)
                    if removed:
                        logger.warning(f"중복 제거 수행: keys={dedupe_keys}, 제거={removed} 행")
                        print(f"    중복 제거: {removed:,} 행 제거됨")
                    else:
                        logger.info(f"중복 없음 확인: keys={dedupe_keys}")
                else:
                    logger.info(f"중복 제거 스킵: dedupe key 부족 (columns={dedupe_keys})")
    
    failed = failed_final
    print(f"최종 실패: {len(failed)}개")
    logger.info(f"최종 실패 티커: {len(failed)}개")

# 최종 결과 백업 저장
if not df_raw.empty:
    backup_path = DATA_DIR / f'raw_data_backup_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
    df_raw.to_csv(backup_path, index=False)
    print(f"\n  백업 저장 완료: {backup_path.name}")
    logger.info(f"백업 저장: {backup_path}")
    
    # 실패 티커 목록 저장
    if failed:
        failed_file = LOG_DIR / f'failed_tickers_{datetime.now().strftime("%Y%m%d_%H%M%S")}.txt'
        with open(failed_file, 'w') as f:
            f.write('\n'.join(failed))
        print(f"  실패 티커 목록 저장: {failed_file.name}")
        logger.info(f"실패 티커 목록 저장: {failed_file}")

2026-01-13 13:13:31,994 - INFO - 실패 티커 재시도 시작: 1개



실패한 티커 재수집 시도
재시도할 티커 수: 1
더 작은 배치 크기(5)로 재시도 중



2026-01-13 13:13:36,998 - INFO - 총 1개 배치로 분할 (배치 크기: 5)
 데이터 수집 중: 100%|██████████| 1/1 [00:00<00:00, 46.59it/s]
2026-01-13 13:13:37,023 - INFO - 총 1개 배치 데이터 병합 중...
2026-01-13 13:13:37,029 - INFO - 최종 데이터: (17369, 7)
2026-01-13 13:13:37,029 - INFO - 컬럼: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Company']
2026-01-13 13:13:37,041 - INFO - 재수집 성공: 1개
2026-01-13 13:13:37,041 - INFO - 최종 실패 티커: 0개


    배치 0 캐시에서 로드

  재수집 성공: 1개
최종 실패: 0개


2026-01-13 13:13:39,530 - INFO - 백업 저장: /Users/yu_seok/Documents/workspace/nbCamp/Project/Yahoo Finance/Data_set/raw_data_backup_20260113_131337.csv



  백업 저장 완료: raw_data_backup_20260113_131337.csv


## Section 3. 티커 메타데이터 수집

In [46]:
# Section 3 전제 조건 체크
print("\n" + "=" * 60)
print("Section 3: 티커 메타데이터 수집")
print("=" * 60)

# 티커 목록 확인
if 'tickers' not in locals() or len(tickers) == 0:
    logger.error("티커 목록이 없습니다")
    raise SystemExit("  티커 목록이 없습니다. Section 1을 먼저 실행하세요.")

print(f"  티커 목록 확인: {len(tickers)}개")
logger.info("Section 3 시작 - 메타데이터 수집")

2026-01-13 13:13:39,536 - INFO - Section 3 시작 - 메타데이터 수집



Section 3: 티커 메타데이터 수집
  티커 목록 확인: 481개


In [47]:
# 티커별 섹터/산업 정보 수집 (개선 버전)
def fetch_ticker_metadata(ticker_list, batch_size=10, max_retries=5):
    """
    개선된 메타데이터 수집 함수
    """
    metadata = {}
    failed_tickers = []
    
    # 캐시 파일 확인
    cache_file = DATA_DIR / 'metadata_cache.json'
    if cache_file.exists():
        try:
            with open(cache_file, 'r', encoding='utf-8') as f:
                metadata = json.load(f)
            logger.info(f"캐시에서 {len(metadata)}개 메타데이터 로드")
            print(f"  캐시에서 {len(metadata)}개 메타데이터 로드됨")
            
            # 이미 수집된 티커 제외
            ticker_list = [t for t in ticker_list if t not in metadata]
            print(f"→ 새로 수집할 티커: {len(ticker_list)}개")
            logger.info(f"새로 수집할 티커: {len(ticker_list)}개")
        except Exception as e:
            logger.warning(f"캐시 로드 실패: {str(e)}")
    
    if len(ticker_list) == 0:
        logger.info("모든 메타데이터가 캐시에 존재함")
        return metadata, failed_tickers
    
    chunks = [ticker_list[i:i + batch_size] for i in range(0, len(ticker_list), batch_size)]
    
    for chunk_idx, chunk in enumerate(tqdm(chunks, desc="📋 메타데이터 수집 중")):
        for symbol in chunk:
            if symbol in metadata:
                continue
            
            # 재시도 로직
            for attempt in range(max_retries):
                try:
                    ticker = yf.Ticker(symbol)
                    info = ticker.info
                    
                    metadata[symbol] = {
                        'Sector': info.get('sector', 'Unknown'),
                        'Industry': info.get('industry', 'Unknown')
                    }
                    break
                
                except Exception as e:
                    if attempt == max_retries - 1:
                        logger.error(f"메타데이터 수집 실패 - {symbol}: {str(e)}")
                        failed_tickers.append(symbol)
                        metadata[symbol] = {'Sector': 'Error', 'Industry': 'Error'}
                    else:
                        wait_time = 2 ** attempt
                        time.sleep(wait_time)
            
            time.sleep(1)  # 티커 간 대기
        
        # 5개 배치마다 캐시 저장
        if chunk_idx % 5 == 0 or chunk_idx == len(chunks) - 1:
            try:
                with open(cache_file, 'w', encoding='utf-8') as f:
                    json.dump(metadata, f, ensure_ascii=False, indent=2)
                logger.info(f"캐시 저장: {len(metadata)}개")
            except Exception as e:
                logger.error(f"캐시 저장 실패: {str(e)}")
        
        time.sleep(2)  # 배치 간 대기
    
    # 최종 캐시 저장
    try:
        with open(cache_file, 'w', encoding='utf-8') as f:
            json.dump(metadata, f, ensure_ascii=False, indent=2)
        logger.info(f"최종 캐시 저장: {len(metadata)}개")
    except Exception as e:
        logger.error(f"최종 캐시 저장 실패: {str(e)}")
    
    return metadata, failed_tickers

In [48]:
# 메타데이터 수집 실행
metadata, failed_meta = fetch_ticker_metadata(tickers, batch_size=10, max_retries=5)
metadata_df = pd.DataFrame.from_dict(metadata, orient='index').reset_index()
metadata_df.columns = ['Company', 'Sector', 'Industry']

print("\n" + "=" * 60)
print("메타데이터 수집 결과")
print("=" * 60)
print(f"  성공: {len(metadata) - len(failed_meta)}개")
print(f"    실패: {len(failed_meta)}개")
print(f"\n섹터 분포 (상위 10개):")
print(metadata_df['Sector'].value_counts().head(10))

logger.info(f"메타데이터 수집 완료 - 성공: {len(metadata) - len(failed_meta)}개, 실패: {len(failed_meta)}개")

📋 메타데이터 수집 중: 100%|██████████| 49/49 [13:15<00:00, 16.23s/it]
2026-01-13 13:26:54,997 - INFO - 최종 캐시 저장: 481개
2026-01-13 13:26:55,008 - INFO - 메타데이터 수집 완료 - 성공: 481개, 실패: 0개



메타데이터 수집 결과
  성공: 481개
    실패: 0개

섹터 분포 (상위 10개):
Sector
Financial Services        82
Technology                77
Healthcare                56
Industrials               56
Consumer Cyclical         45
Energy                    38
Consumer Defensive        31
Communication Services    27
Basic Materials           26
Utilities                 24
Name: count, dtype: int64


In [49]:
# 원본 데이터에 메타데이터 매핑
print("\n" + "=" * 60)
print("메타데이터 매핑")
print("=" * 60)

# 전제 조건 확인
if 'df_raw' not in locals() or df_raw.empty:
    logger.error("df_raw가 비어있습니다")
    raise SystemExit("  df_raw가 비어있습니다. Section 2를 먼저 성공적으로 실행하세요.")

if 'Company' not in df_raw.columns:
    logger.error("df_raw에 'Company' 컬럼이 없습니다")
    raise SystemExit(f"  df_raw에 'Company' 컬럼이 없습니다. 현재 컬럼: {df_raw.columns.tolist()}")

if metadata_df.empty:
    logger.error("metadata_df가 비어있습니다")
    raise SystemExit("  metadata_df가 비어있습니다. 메타데이터를 먼저 수집하세요.")

# 매핑 전 정보
print(f"df_raw Shape: {df_raw.shape}")
print(f"metadata_df Shape: {metadata_df.shape}")

# 매핑 실행
df_raw = df_raw.merge(metadata_df, on='Company', how='left')

# 매핑 후 정보
print(f"\n  매핑 완료")
print(f"최종 Shape: {df_raw.shape}")
print(f"최종 컬럼: {df_raw.columns.tolist()}")

# 샘플 데이터 출력
if all(col in df_raw.columns for col in ['Company', 'Date', 'Close', 'Sector', 'Industry']):
    print(f"\n샘플 데이터:")
    print(df_raw[['Company', 'Date', 'Close', 'Sector', 'Industry']].head(10))

logger.info(f"메타데이터 매핑 완료 - 최종 Shape: {df_raw.shape}")

2026-01-13 13:26:55,137 - INFO - 메타데이터 매핑 완료 - 최종 Shape: (860882, 9)



메타데이터 매핑
df_raw Shape: (860882, 7)
metadata_df Shape: (481, 3)

  매핑 완료
최종 Shape: (860882, 9)
최종 컬럼: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Company', 'Sector', 'Industry']

샘플 데이터:
  Company       Date      Close      Sector                Industry
0       A 2018-11-29  67.863747  Healthcare  Diagnostics & Research
1       A 2018-11-30  68.603371  Healthcare  Diagnostics & Research
2       A 2018-12-03  70.803230  Healthcare  Diagnostics & Research
3       A 2018-12-04  69.134399  Healthcare  Diagnostics & Research
4       A 2018-12-06  68.186165  Healthcare  Diagnostics & Research
5       A 2018-12-07  66.612114  Healthcare  Diagnostics & Research
6       A 2018-12-10  66.934502  Healthcare  Diagnostics & Research
7       A 2018-12-11  67.370697  Healthcare  Diagnostics & Research
8       A 2018-12-12  68.034424  Healthcare  Diagnostics & Research
9       A 2018-12-13  67.456024  Healthcare  Diagnostics & Research


## Section 4. 데이터 품질 검증 (QA)

In [50]:
# Section 4 전제 조건 체크
print("\n" + "=" * 60)
print("Section 4: 데이터 품질 검증")
print("=" * 60)

if 'df_raw' not in locals() or df_raw.empty:
    logger.error("df_raw가 비어있습니다")
    print("  수집된 데이터가 없습니다. Section 2를 먼저 성공적으로 실행하세요.")
    print("   또는 기존 저장된 파일을 로드하세요:")
    print(f"   df_raw = pd.read_csv('{DATA_DIR / 'raw_data_backup_*.csv'}')")
    raise SystemExit("데이터 필요")

# 필수 컬럼 체크
required_cols = ['Date', 'Company', 'Open', 'High', 'Low', 'Close', 'Volume']
missing_cols = [col for col in required_cols if col not in df_raw.columns]
if missing_cols:
    logger.error(f"필수 컬럼 누락: {missing_cols}")
    raise SystemExit(f"  필수 컬럼 누락: {missing_cols}")

print("  데이터 검증 통과")
print(f"  Shape: {df_raw.shape}")
print(f"  기업 수: {df_raw['Company'].nunique()}")
print(f"  기간: {df_raw['Date'].min()} ~ {df_raw['Date'].max()}")
logger.info("Section 4 시작 - 데이터 품질 검증")

2026-01-13 13:26:55,188 - INFO - Section 4 시작 - 데이터 품질 검증



Section 4: 데이터 품질 검증
  데이터 검증 통과
  Shape: (860882, 9)
  기업 수: 480
  기간: 2018-11-29 00:00:00 ~ 2026-01-12 00:00:00


In [51]:
# 기업별 관측일 수 통계
company_stats = df_raw.groupby('Company').agg({
    'Date': ['count', 'min', 'max']
}).reset_index()
company_stats.columns = ['Company', 'Obs_Count', 'Start_Date', 'End_Date']
company_stats['Days_Span'] = (company_stats['End_Date'] - company_stats['Start_Date']).dt.days

print("\n" + "=" * 60)
print("기업별 관측일 수 통계")
print("=" * 60)
print(company_stats[['Obs_Count', 'Days_Span']].describe())

# 부족한 데이터 기업
threshold = company_stats['Obs_Count'].max() * 0.8
partial_data = company_stats[company_stats['Obs_Count'] < threshold]
print(f"\n   관측일 부족 기업: {len(partial_data)}개 (80% 미만)")
if len(partial_data) > 0:
    print("\n상위 10개:")
    print(partial_data[['Company', 'Obs_Count', 'Start_Date']].sort_values('Obs_Count').head(10))

logger.info(f"관측일 부족 기업: {len(partial_data)}개")

2026-01-13 13:26:55,230 - INFO - 관측일 부족 기업: 471개



기업별 관측일 수 통계
         Obs_Count    Days_Span
count   480.000000   480.000000
mean   1793.504167  2556.318750
std     286.821593   208.694132
min     584.000000   851.000000
25%    1788.000000  2601.000000
50%    1788.000000  2601.000000
75%    1788.000000  2601.000000
max    3576.000000  2601.000000

   관측일 부족 기업: 471개 (80% 미만)

상위 10개:
    Company  Obs_Count Start_Date
37      ARM        584 2023-09-14
252    KVUE        675 2023-05-04
187    GEHC        770 2022-12-15
269    MBLY        805 2022-10-26
208     HLN        871 2022-07-25
86      CEG        999 2022-01-19
314      NU       1026 2021-12-09
188     GFS       1055 2021-10-28
101    COIN       1193 2021-04-14
106    CPNG       1216 2021-03-11


In [52]:
# OHLC 데이터 이상치 검증
def validate_ohlc(df, tolerance=0.01):
    """OHLC 데이터 이상치 검증"""
    anomalies = []
    
    # 1. 0값 또는 음수
    for col in ['Open', 'High', 'Low', 'Close']:
        if col not in df.columns:
            continue
        zero_or_neg = df[df[col] <= 0]
        for _, row in zero_or_neg.iterrows():
            anomalies.append({
                'Company': row['Company'],
                'Date': row['Date'],
                'Type': f'{col} <= 0',
                'Value': row[col]
            })
    
    # 2. High < Low 역전
    if 'High' in df.columns and 'Low' in df.columns:
        inverted = df[df['High'] < (df['Low'] - tolerance)]
        for _, row in inverted.iterrows():
            anomalies.append({
                'Company': row['Company'],
                'Date': row['Date'],
                'Type': 'High < Low',
                'Value': f"H:{row['High']:.2f}, L:{row['Low']:.2f}"
            })
    
    # 3. Open/Close가 High/Low 범위 밖
    if all(col in df.columns for col in ['Open', 'High', 'Low', 'Close']):
        out_of_range = df[
            (df['Open'] > df['High'] + tolerance) | 
            (df['Open'] < df['Low'] - tolerance) |
            (df['Close'] > df['High'] + tolerance) | 
            (df['Close'] < df['Low'] - tolerance)
        ]
        for _, row in out_of_range.iterrows():
            anomalies.append({
                'Company': row['Company'],
                'Date': row['Date'],
                'Type': 'Open/Close out of H/L range',
                'Value': f"O:{row['Open']:.2f}, H:{row['High']:.2f}, L:{row['Low']:.2f}, C:{row['Close']:.2f}"
            })
    
    return pd.DataFrame(anomalies)

anomaly_df = validate_ohlc(df_raw)

print("\n" + "=" * 60)
print("OHLC 이상치 검증")
print("=" * 60)
if len(anomaly_df) == 0:
    print("  이상치 없음")
    logger.info("OHLC 이상치 없음")
else:
    print(f"   총 이상치: {len(anomaly_df)}건")
    print(f"\n유형별 집계:")
    print(anomaly_df['Type'].value_counts())
    logger.warning(f"OHLC 이상치: {len(anomaly_df)}건")
    
    # 이상치 저장
    anomaly_file = QA_DIR / 'ohlc_anomalies.csv'
    anomaly_df.to_csv(anomaly_file, index=False)
    print(f"\n  이상치 저장: {anomaly_file.name}")

2026-01-13 13:26:55,252 - WARNING - OHLC 이상치: 2건



OHLC 이상치 검증
   총 이상치: 2건

유형별 집계:
Type
Open/Close out of H/L range    2
Name: count, dtype: int64

  이상치 저장: ohlc_anomalies.csv


In [53]:
# Dividends & Stock Splits 이벤트 확인
print("\n" + "=" * 60)
print("배당 및 주식 분할 이벤트")
print("=" * 60)

# 배당 이벤트
if 'Dividends' in df_raw.columns:
    dividend_events = df_raw[df_raw['Dividends'] > 0]
    print(f"\n배당 이벤트:")
    print(f"  배당 기업 수: {dividend_events['Company'].nunique()}개")
    print(f"  총 배당 이벤트: {len(dividend_events)}건")
    logger.info(f"배당 이벤트: {len(dividend_events)}건")
else:
    print("\n배당 데이터 없음")

# 주식 분할 이벤트
if 'Stock Splits' in df_raw.columns:
    split_events = df_raw[df_raw['Stock Splits'] > 0]
    print(f"\n주식 분할 이벤트:")
    print(f"  분할 기업 수: {split_events['Company'].nunique()}개")
    print(f"  총 분할 이벤트: {len(split_events)}건")
    logger.info(f"주식 분할 이벤트: {len(split_events)}건")
    
    if len(split_events) > 0:
        print(f"\n상세 (상위 10건):")
        print(split_events[['Company', 'Date', 'Stock Splits']].head(10))
else:
    print("\n주식 분할 데이터 없음")


배당 및 주식 분할 이벤트

배당 데이터 없음

주식 분할 데이터 없음


## Section 5. 데이터 정제

In [54]:
# Section 5 전제 조건 체크
print("\n" + "=" * 60)
print("Section 5: 데이터 정제")
print("=" * 60)

if 'df_raw' not in locals() or df_raw.empty:
    logger.error("df_raw가 비어있습니다")
    raise SystemExit("  데이터가 없습니다. Section 2-4를 먼저 실행하세요.")

print("  데이터 확인 완료")
logger.info("Section 5 시작 - 데이터 정제")

2026-01-13 13:26:55,262 - INFO - Section 5 시작 - 데이터 정제



Section 5: 데이터 정제
  데이터 확인 완료


In [55]:
# OHLC 범위 오류 수정
def clean_ohlc_violations(df, tolerance=0.01):
    """OHLC 범위 오류 수정"""
    df_cleaned = df.copy()
    cleaning_log = []
    
    # 필수 컬럼 확인
    if not all(col in df.columns for col in ['Open', 'High', 'Low', 'Close']):
        logger.warning("OHLC 컬럼 일부 누락, 정제 스킵")
        return df_cleaned, pd.DataFrame()
    
    # Open > High
    mask = df_cleaned['Open'] > (df_cleaned['High'] + tolerance)
    if mask.sum() > 0:
        for idx in df_cleaned[mask].index:
            row = df_cleaned.loc[idx]
            cleaning_log.append({
                'Index': idx, 'Company': row['Company'], 'Date': row['Date'],
                'Type': 'Open > High', 'Original_High': row['High'], 'Corrected_High': row['Open']
            })
        df_cleaned.loc[mask, 'High'] = df_cleaned.loc[mask, 'Open']
    
    # Open < Low
    mask = df_cleaned['Open'] < (df_cleaned['Low'] - tolerance)
    if mask.sum() > 0:
        for idx in df_cleaned[mask].index:
            row = df_cleaned.loc[idx]
            cleaning_log.append({
                'Index': idx, 'Company': row['Company'], 'Date': row['Date'],
                'Type': 'Open < Low', 'Original_Low': row['Low'], 'Corrected_Low': row['Open']
            })
        df_cleaned.loc[mask, 'Low'] = df_cleaned.loc[mask, 'Open']
    
    # Close > High
    mask = df_cleaned['Close'] > (df_cleaned['High'] + tolerance)
    if mask.sum() > 0:
        for idx in df_cleaned[mask].index:
            row = df_cleaned.loc[idx]
            cleaning_log.append({
                'Index': idx, 'Company': row['Company'], 'Date': row['Date'],
                'Type': 'Close > High', 'Original_High': row['High'], 'Corrected_High': row['Close']
            })
        df_cleaned.loc[mask, 'High'] = df_cleaned.loc[mask, 'Close']
    
    # Close < Low
    mask = df_cleaned['Close'] < (df_cleaned['Low'] - tolerance)
    if mask.sum() > 0:
        for idx in df_cleaned[mask].index:
            row = df_cleaned.loc[idx]
            cleaning_log.append({
                'Index': idx, 'Company': row['Company'], 'Date': row['Date'],
                'Type': 'Close < Low', 'Original_Low': row['Low'], 'Corrected_Low': row['Close']
            })
        df_cleaned.loc[mask, 'Low'] = df_cleaned.loc[mask, 'Close']
    
    return df_cleaned, pd.DataFrame(cleaning_log)

df_clean, cleaning_log = clean_ohlc_violations(df_raw, tolerance=0.01)

print("\n" + "=" * 60)
print("OHLC 범위 오류 수정")
print("=" * 60)
print(f"수정 건수: {len(cleaning_log)}")

if len(cleaning_log) > 0:
    print(f"\n수정 유형별:")
    print(cleaning_log['Type'].value_counts())
    
    cleaning_file = QA_DIR / 'ohlc_cleaning_log.csv'
    cleaning_log.to_csv(cleaning_file, index=False)
    print(f"\n  로그 저장: {cleaning_file.name}")
    logger.info(f"OHLC 정제: {len(cleaning_log)}건 수정")
else:
    print("  수정할 항목 없음")
    logger.info("OHLC 정제: 수정 항목 없음")

2026-01-13 13:26:55,331 - INFO - OHLC 정제: 2건 수정



OHLC 범위 오류 수정
수정 건수: 2

수정 유형별:
Type
Open < Low    2
Name: count, dtype: int64

  로그 저장: ohlc_cleaning_log.csv


In [56]:
# 극단 변화 플래그 추가
def add_extreme_change_flag(df, threshold=0.5):
    """극단적 일간 변동 플래그 추가"""
    df = df.copy()
    
    if 'Close' not in df.columns or 'Company' not in df.columns:
        logger.warning("필수 컬럼 없음, 극단 변화 플래그 스킵")
        return df
    
    df['Daily_Change_Temp'] = df.groupby('Company')['Close'].pct_change().abs()
    df['Is_Extreme_Change'] = df['Daily_Change_Temp'] > threshold
    df = df.drop(columns=['Daily_Change_Temp'])
    return df

df_clean = add_extreme_change_flag(df_clean, threshold=0.5)
extreme_count = df_clean['Is_Extreme_Change'].sum() if 'Is_Extreme_Change' in df_clean.columns else 0

print(f"\n  극단 변화 플래그 추가: {extreme_count}건 (삭제하지 않고 보존)")
logger.info(f"극단 변화 플래그: {extreme_count}건")

2026-01-13 13:26:55,440 - INFO - 극단 변화 플래그: 17건



  극단 변화 플래그 추가: 17건 (삭제하지 않고 보존)


In [57]:
# 재검증
print("\n" + "=" * 60)
print("정제 후 재검증")
print("=" * 60)

anomaly_after = validate_ohlc(df_clean, tolerance=0.01)

if len(anomaly_after) == 0:
    print("  OHLC 이상치 없음")
    logger.info("정제 후 OHLC 이상치 없음")
else:
    print(f"   남은 이상치: {len(anomaly_after)}건")
    print(anomaly_after['Type'].value_counts())
    logger.warning(f"정제 후 남은 이상치: {len(anomaly_after)}건")


정제 후 재검증
  OHLC 이상치 없음

2026-01-13 13:26:55,458 - INFO - 정제 후 OHLC 이상치 없음


In [ ]:
# 정제된 데이터 저장 (adjusted 파일)
print("\n" + "=" * 60)
print("정제된 데이터 저장")
print("=" * 60)

print(f"Shape: {df_clean.shape}")
if 'Date' in df_clean.columns:
    print(f"기간: {df_clean.min()['Date']} ~ {df_clean.max()['Date']}")
if 'Company' in df_clean.columns:
    print(f"기업 수: {df_clean['Company'].nunique()}")

try:
    output_path = DATA_DIR / 'stock_daily_master_adjusted.csv'
    df_clean.to_csv(output_path, index=False)
    
    print(f"\n✓ 저장 완료: {output_path.name}")
    print(f"  파일 크기: {output_path.stat().st_size / (1024*1024):.2f} MB")
    
    logger.info(f"정제된 데이터 저장 완료: {output_path}")
    logger.info(f"Shape: {df_clean.shape}")
    
    print("\n샘플 데이터 (처음 5행):")
    display_cols = ['Company', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume']
    display_cols = [col for col in display_cols if col in df_clean.columns]
    print(df_clean[display_cols].head())
    
except Exception as e:
    logger.error(f"데이터 저장 실패: {str(e)}")
    print(f" 저장 중 에러: {str(e)}")
    raise

print("\n" + "=" * 60)
print(" 데이터 수집 및 정제 완료")
print("=" * 60)
print("\n다음 단계: 04_Derived_Variable.ipynb를 실행하여 파생변수를 계산하세요.")